In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train_data = pd.read_csv("/train.csv")
test_data = pd.read_csv("/test.csv")
test_label = pd.read_csv("/test_labels.csv").replace(-1,0)


In [3]:
test_data = pd.merge(test_data, test_label, how = 'left', on = 'id')
total_data = pd.concat([train_data, test_data])
total_data.sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
48876,82ad6839c662694d,"""\n\nOrphaned fair use image (:Image:Kivul.JPE...",0,0,0,0,0,0
70098,74b1778abc2c9f0b,"""You mean got her hands dirty """"fucking"""" for ...",0,0,0,0,0,0
72477,c1f3f92a76f424a2,"Well, I guess I was a bit unprepared for the a...",0,0,0,0,0,0
10365,1155e98431c2cf26,"Anyway, I didn't mean it in a bad way, OK?!",0,0,0,0,0,0
112816,5b828e9258a3098c,". If no reliable text can be found, an image ...",0,0,0,0,0,0
22086,24de676691e004e3,Casey needs to have her own page,0,0,0,0,0,0
147153,f5fde486a4278e16,Why Are you such a Fag \n\n Your life consists...,0,0,0,0,0,0
146482,2cc228a2e4d67134,I'm posting on your talk page,0,0,0,0,0,0
47741,4f385c51d9293855,Any ideas to why eminems new album the re-up a...,0,0,0,0,0,0
138064,e6cb3b078b8eb1a7,== Rajput == \n\n If you're going to unprotect...,0,0,0,0,0,0


In [4]:
total_data['severe_toxic'] = total_data.severe_toxic * 2
#otal_data['obscene'] = total_data.obscene * 2
#total_data['insult'] = total_data.obscene * 3
#total_data['threat'] = total_data.threat * 4
#total_data['identity_hate'] = total_data.identity_hate * 5
total_data['y'] = (total_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)).astype(int)
#total_data['y'] = total_data['y']/total_data['y'].max()
total_data = total_data[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

In [5]:
total_data.sample(10)
total_data['y'].value_counts()

0    290267
1      8202
3      6117
2      4969
4      1392
5      1274
6       469
7        45
Name: y, dtype: int64

In [6]:
total_data = pd.concat([total_data[total_data['y'] > 0], total_data[total_data['y'] == 0].sample(int(len(total_data[total_data['y'] > 0])*1.5))], axis = 0).sample(frac = 1)

In [7]:
def clean(data, col):  # Replace each occurrence of pattern/regex in the Series/Index

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')  
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    
    return data  # the function returns the processed value

In [8]:
total_data = clean(total_data,'text')

In [9]:
total_data.sample(10)

,text,y
138330,Oh of fuck you asshole die im going to kill yo...,6
128393,""" \n :::He is also engaging in what may be the...",0
149726,HI\ \n \n \n I WANT TO HAVE SEX WITH U \n \n \...,1
67967,"Cumulus cloud, you should respond of your disr...",0
32131,"Welcome, Korean assh * le ' s page. \n Why don...",1
129553,"Cement mixer redirects here, but isn ' t liste...",0
313,:::::::::Moi ? Ego ? I am mortified that you c...,2
95960,""" \n \n That is not a insult. That is a realit...",1
142917,Stop reverting my edits \n \n Please stop reve...,1
9345,"""Obama is NOT """"African American"""" you jackass...",3


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(total_data['text'])

In [11]:
text_counts.shape

(56170, 95962)

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer()
total_tfidf= tf.fit_transform(text_counts)
total_tfidf.shape

(56170, 95962)

In [13]:
from sklearn.naive_bayes import MultinomialNB
#model = MultinomialNB()
from sklearn.linear_model import Ridge
model = Ridge()
model.fit(total_tfidf, total_data['y'])

Ridge()

In [20]:
model.score(total_tfidf, total_data['y'])

0.7326381541039744

In [14]:
val_data = pd.read_csv("/validation_data.csv")
val_data = clean(val_data,'less_toxic')
val_data = clean(val_data,'more_toxic')
less_toxic = tf.transform(cv.transform(val_data['less_toxic']))
more_toxic = tf.transform(cv.transform(val_data['more_toxic']))
pred_less = model.predict(less_toxic)
pred_more = model.predict(more_toxic)


In [15]:
(pred_more > pred_less).mean()

0.678158628935831

In [16]:
final_data = pd.read_csv("/comments_to_score.csv")
final_data.sample(10)


,comment_id,text
4548,255830882,DUDE< I DO IT BECAUSE YOUR MUM SCREAMS TOOOOO ...
2598,142194577,well you can suck my balls gay fuck. Wikipedia...
434,33481190,"""\nBan one side of an argument by a bullshit n..."
7084,459258858,"""\nHow can I know? His name is """"Ramanatruth""""..."
5840,345862997,It's aganist any logic to try to separate the ...
235,22448624,"""puh-leez, Coulter herself said she was wrong ..."
6101,367593084,block user:sco1996\nDear Syrthiss.\nsco1996 is...
5829,344828605,u think i get impressed by some f@ck!ng swedi...
5791,342418929,"""Because Peter Symonds is just as big an idiot..."
1127,62118800,"kindness\n\nWow, your compassion sure disapper..."


In [17]:
final_data = clean(final_data,'text')
#final_token = RegexpTokenizer(r'[a-zA-Z0-9]+')
#cv_final = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = final_token.tokenize)
text_countsfinal= cv.transform(final_data['text'])
text_countsfinal.shape
#tf_final = TfidfTransformer()
#final_tfidf= tf.transform(text_countsfinal)


(7537, 95962)

In [18]:
final_tfidf= tf.transform(text_countsfinal)
final_tfidf.shape
preds = model.predict(final_tfidf)
print(type(preds))

<class 'numpy.ndarray'>


In [19]:
#preds = model.predict(final_tfidf)
final_data['score'] =  preds
final_data[['comment_id', 'score']].to_csv("submission.csv", index=False)